In [1]:
import torch
import sys
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel

### LM

In [2]:
# Load pre-trained model (weights)
with torch.no_grad():
        model = GPT2LMHeadModel.from_pretrained('gpt2')
        model.eval()
# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
 
def score(sentence):
    tokenize_input = tokenizer.encode(sentence)
    tensor_input = torch.tensor([tokenize_input])
    rs=model(tensor_input, labels=tensor_input)
    loss = rs[0]
    return np.exp(loss.detach().numpy()), rs
 

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
loss, rs = score('this is what i dont like at 460 jtgle')

loss, len(rs), rs[0], rs[1].shape, len(rs[2])

(1324.6301,
 3,
 tensor(7.1889, grad_fn=<NllLossBackward>),
 torch.Size([1, 11, 50257]),
 12)

In [4]:
sum(rs[2]).shape

torch.Size([2, 1, 12, 11, 64])

## pretrained Embedding

In [1]:
from transformers import GPT2Tokenizer, GPT2Model
import torch

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

def model_out(s="Hello, my dog is cute"):
    inputs = tokenizer(s, return_tensors="pt")
    outputs = model(**inputs)
    return outputs

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
out=model_out("fuck you")
#last_hidden_states = out.last_hidden_state

len(out), out[0].shape, out[1][0].shape

(2, torch.Size([1, 2, 768]), torch.Size([2, 1, 12, 2, 64]))

In [3]:
sum(out[0]).shape

torch.Size([2, 768])

In [12]:
out[0][0].shape[0]

2

In [1]:
#sum(out[0][0])/2

# lstm

In [2]:
def build_model(max_features, maxlen, gpt2_len):
    model = Sequential()
    
    #model.add(Embedding(max_features, 480, input_length=maxlen + gpt2_len))
    
    model.add(Bidirectional(LSTM(240, return_sequences = True)))
    
    model.add(Bidirectional(LSTM(160, return_sequences = True)))
    
    model.add(Bidirectional(LSTM(80)))
    
    model.add(Dropout(drop_out))

    model.add(Dense(2, activation='sigmoid'))
    
    model.compile('adam', 'sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

#model = build_model(max_features=max_features, maxlen= maxlen, gpt2_len=gpt2_len)
#model.summary()

In [11]:
!ls "../data/datast-v2"

train-token-df.csv  train-trail-tokens.zip  trial-token-df.csv


In [12]:
import pandas as pd

trial = pd.read_csv("../data/datast-v2/trial-token-df.csv")
train = pd.read_csv("../data/datast-v2/train-token-df.csv").dropna()

train.head(3)

,index,text,grand-truth,taboo-words,token,token-tag,token-iob,token-span,class
0,0,Another violent and aggressive immigrant killi...,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",['violent and aggressive immigrant'],Another,DT,O,"[0, 1, 2, 3, 4, 5, 6]",normal
1,0,Another violent and aggressive immigrant killi...,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",['violent and aggressive immigrant'],violent,JJ,O,"[8, 9, 10, 11, 12, 13, 14]",toxic
2,0,Another violent and aggressive immigrant killi...,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",['violent and aggressive immigrant'],and,CC,O,"[16, 17, 18]",toxic


In [1]:
from transformers import GPT2Tokenizer, GPT2Model
import torch
from tqdm import tqdm

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

__vectors__ = {}

def get_gpt2_vector(word):
    try:
        vec = __vectors__[word]
        return vec
    except:
        inputs = tokenizer(word, return_tensors="pt")
        outputs = model(**inputs)
        #outputs = model(**inputs)[0][0]
        #vec = sum(outputs.detach().numpy())/outputs.shape[0]
        vec = np.average(outputs[0].detach().numpy(), axis=1)[0]
        __vectors__[word] = vec
        return vec

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
import numpy as np

In [4]:
xx = get_gpt2_vector("ali")


In [5]:
xx.shape

(768,)

In [14]:
X_train = train['token'].tolist()
X_train_label = train['class'].tolist()
X_train_span = train['token-span'].tolist()

X_test = trial['token'].tolist()
X_test_label = trial['class'].tolist()
X_test_span = trial['token-span'].tolist()

print("Train size:", len(X_train))
print("Test size:", len(X_test))

Train size: 284169
Test size: 24027


In [15]:
size = 100

X_train = X_train[:size]
y_train = X_train_label[:size]

X_test = X_test[:size]
y_test = X_test_label[:size]

In [17]:
import numpy as np

X_train_vec = []
for w in X_train:
    X_train_vec.append(get_gpt2_vector(w))
X_train_vec = np.array(X_train_vec)

X_test_vec = []
for w in X_test:
    X_test_vec.append(get_gpt2_vector(w))
X_test_vec = np.array(X_test_vec)

In [18]:
X_train_vec.shape, X_test_vec.shape

((100, 768), (100, 768))

In [122]:
from keras.preprocessing import sequence
from keras.models import Sequential, Input
from keras.layers import Dense, Dropout, Embedding, Bidirectional, LSTM
from keras.callbacks import TensorBoard

In [129]:
def build_model(input_shape):
    model = Sequential()
    
    model.add(Bidirectional(LSTM(10, return_sequences = True, input_shape=input_shape)))
    
    #model.add(Bidirectional(LSTM(160, return_sequences = True)))
    
    #model.add(Bidirectional(LSTM(80, return_sequences=False)))
    
    #model.add(Dropout(0.5))

    model.add(Dense(2, activation='sigmoid'))
    
    model.compile('adam', 'sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

#model = build_model(max_features=max_features, maxlen= maxlen, gpt2_len=gpt2_len)
#model.summary()

In [134]:
X_train_vec_ = X_train_vec.reshape(50, 2, 768)
y = np.array([1 if y=='normal' else 0 for y in y_train]).reshape(50,2,1)
y_train_ = np.array([1 if y=='normal' else 0 for y in y_train]).reshape(50, 2)
X_test_vec_ = X_test_vec.reshape(50, 2, 768)
y_test_ = np.array([1 if y=='normal' else 0 for y in y_test]).reshape(50, 2)

model = build_model((1, 2,768))
model.build((1, 2,768))
print(model.summary())


history = model.fit(X_train_vec_, y,
                  batch_size=1,
                  epochs=5,
                  validation_split=0,
                  verbose=1)

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_87 (Bidirectio (1, 2, 480)               1937280   
_________________________________________________________________
bidirectional_88 (Bidirectio (1, 2, 320)               820480    
_________________________________________________________________
dense_32 (Dense)             (1, 2, 2)                 642       
Total params: 2,758,402
Trainable params: 2,758,402
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5


KeyboardInterrupt: 

In [12]:
for i in range(1, int(284169/2)):
    if 284169%i == 0:
        print(i)

1
3
94723


In [13]:
284169/94723

3.0